In [2]:
import os, sys, shutil
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
sys.path.append("../src")
from utils import *
from data_utils import *
from autoencoder import AutoEncoder
from gnet8_encoder import GNet8_Encoder
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import random
os.chdir("..")

/export/raid1/home/kneel027/miniconda3/envs/SS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_train, x_val, x_test, _, _, images, _ = load_nsd(vector="images", subject=1, loader=False, average=False, nest=True, return_sessions=False, normalized=False, split_val=False, GLM_denoise=False)

loading samples: 100%|██████████| 27749/27749 [00:09<00:00, 2813.31it/s] 


Data Shapes... x_train: torch.Size([24979, 15724]), x_val: torch.Size([0]), x_test: torch.Size([982, 3, 15724]), y_train: torch.Size([24979, 541875]), y_val: torch.Size([0]), y_test: torch.Size([982, 541875])


In [3]:
train_norm = torch.zeros_like(x_train)
test_norm = torch.zeros_like(x_test)
print(x_train.shape)
for i in range(x_train.shape[1]):
    voxel_mean, voxel_std = torch.mean(x_train[:, i]), torch.std(x_train[:, i])  
    if i < 5:
        print(voxel_mean, voxel_std)
    train_norm[:, i] = (x_train[:, i] - voxel_mean) / voxel_std
    test_norm[:, :, i] = (x_test[:, :, i] - voxel_mean) / voxel_std
test_norm = torch.mean(test_norm, axis=1)
print(test_norm.shape)
torch.save(test_norm, "/home/naxos2-raid25/kneel027/home/kneel027/MindEyeV2/subj1test_norm4.pt")

torch.Size([24979, 15724])
tensor(242.8461) tensor(737.2141)
tensor(-56.3801) tensor(482.1737)
tensor(36.5304) tensor(294.0379)
tensor(265.7399) tensor(440.3240)
tensor(431.9478) tensor(446.5452)
torch.Size([982, 15724])


In [2]:
for subject in tqdm(range(1,9)):
    create_whole_region_unnormalized(subject, include_heldout=True)
    create_whole_region_normalized(subject, include_heldout=True)
    

  0%|          | 0/8 [02:18<?, ?it/s]


KeyboardInterrupt: 

In [3]:
process_data(vector="images", include_heldout=True)

processing data: 100%|██████████| 8/8 [16:21<00:00, 122.70s/it]


In [5]:
x_train, x_val, x_test, _, _, images, _ = load_nsd(vector="images", subject=1, loader=False, average=True, nest=False, return_sessions=False, normalized=True, include_heldout=False)
print(x_test.shape, x_test)
torch.save(x_test, "/home/naxos2-raid25/kneel027/home/kneel027/MindEyeV2/subj1test_norm5.pt")

torch.Size([27750, 15724])


loading samples: 100%|██████████| 27750/27750 [00:24<00:00, 1145.46it/s] 


Data Shapes... x_train: torch.Size([21217, 15724]), x_val: torch.Size([3763, 15724]), x_test: torch.Size([982, 15724]), y_train: torch.Size([21217, 541875]), y_val: torch.Size([3763, 541875]), y_test: torch.Size([982, 541875])
torch.Size([982, 15724]) tensor([[ 0.7171, -0.3140, -0.1861,  ..., -0.3072,  0.5959,  0.2289],
        [-0.5112, -0.0537, -0.2221,  ...,  0.9097,  0.2570, -0.1522],
        [ 1.1536,  0.8113,  0.3685,  ...,  0.3797,  0.9819,  0.5518],
        ...,
        [-0.1241, -0.0358,  0.7416,  ..., -0.0606, -0.9639, -0.8716],
        [-0.0182, -0.9197, -0.0622,  ..., -0.0695, -0.4521, -0.2796],
        [-0.9646, -1.4248, -0.6874,  ..., -0.6318, -0.7369, -0.4036]])


In [ ]:
print(x_test)
print(torch.load("/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/data/preprocessed_data/subject1/nsd_general.pt"))


In [45]:
subject=1
stim_descriptions = pd.read_csv('data/nsddata/experiments/nsd/nsd_stim_info_merged.csv', index_col=0)
subj_train = stim_descriptions[(stim_descriptions['subject{}'.format(subject)] != 0) & (stim_descriptions['shared1000'] == True)]
subj_test = stim_descriptions[(stim_descriptions['subject{}'.format(subject)] != 0) & (stim_descriptions['shared1000'] == True)].reset_index()
print(subj_test)
scanIDs = [0]*1000
for i in range(subj_train.shape[0]):
    nsdID = subj_train.iloc[i]['nsdId']
    for j in range(3):
        scanID = subj_train.iloc[i]['subject{}_rep{}'.format(subject, j)] - 1
        if scanID > 37*750:
            shared1000idx = subj_test[subj_test['nsdId'] == nsdID].index
            # print(shared1000idx, shared1000idx[0])
            scanIDs[int(shared1000idx[0])] += 1
print(np.unique(scanIDs, return_counts=True))
indices = [i for i, value in enumerate(scanIDs) if value > 0]
missing = [i for i, value in enumerate(scanIDs) if value == 3]
print(len(indices), indices)
print(scanIDs)
print(missing)

# incomplete_ids = []
# for nsdId, count in scanIDs.items():
#     if count != 3:
#         shared1000idx = subj_test[subj_test['nsdId'] == nsdId].index
#         incomplete_ids.append(shared1000idx.astype(int))
# incomplete_ids = list(np.concatenate(incomplete_ids))
# print(len(incomplete_ids), incomplete_ids)
adjustment_idx = []
swap_margin = 0
for i in range(1000):
    if scanIDs[i] == 3:
        swap_margin += 1
    adjustment_idx.append(i+swap_margin)
print(adjustment_idx)
incrementidx = [j for j in range(1000)]
for i, j in zip(adjustment_idx, incrementidx):
    print(i, j)

     index  cocoId  cocoSplit                         cropBox     loss  nsdId  \
0     2950  262145  train2017  (0, 0, 0.16640625, 0.16640625)  0.09375   2950   
1     2990  262239  train2017    (0, 0, 0.1671875, 0.1671875)  0.10000   2990   
2     3049  262414  train2017            (0, 0, 0.125, 0.125)  0.00000   3049   
3     3077  524646  train2017    (0, 0, 0.1671875, 0.1671875)  0.00000   3077   
4     3146  262690  train2017  (0, 0, 0.16640625, 0.16640625)  0.00000   3146   
..     ...     ...        ...                             ...      ...    ...   
995  72312  515508  train2017            (0, 0, 0.125, 0.125)  0.00000  72312   
996  72510  254130  train2017  (0, 0, 0.16640625, 0.16640625)  0.00000  72510   
997  72605  516634  train2017            (0.125, 0.125, 0, 0)  0.00000  72605   
998  72719  304627  train2017            (0.125, 0.125, 0, 0)  0.00000  72719   
999  72948  517878  train2017      (0, 0, 0.165625, 0.165625)  0.06250  72948   

     flagged  BOLD5000  sha

In [8]:
import h5py
import numpy as np

# Load the file
# data = torch.load("data/preprocessed_data/subject1/nsd_general_large.pt")
# print(data[:,0].mean(), data[:,0].std())
# Save it in HDF5 format
# with h5py.File("/home/naxos2-raid25/kneel027/home/kneel027/MindEyeV2/new_betas/betas_all_subj01_fp32_renorm.hdf5", "w") as f:
#     f.create_dataset("betas", data=data.numpy())
    
    
# Open the HDF5 file
file_path = "/home/naxos2-raid25/kneel027/home/kneel027/MindEyeV2/new_betas/betas_all_subj01_fp32_renorm copy.hdf5"
file_path2 = "/home/naxos2-raid25/kneel027/home/kneel027/MindEyeV2/new_betas/betas_all_subj01_fp32_renorm.hdf5"
with h5py.File(file_path, "r") as f:
    with h5py.File(file_path2, "r") as f2:
    # Access the data or perform operations on the file
    # For example, you can print the keys of the datasets in the file
        are_equal = np.array_equal(f["betas"], f2["betas"])
        print(are_equal)



True
